In [5]:
%pip install pandas
%pip install scikit-learn
%pip install web3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\IoT_Security_Project\AI_Threat_Detection\env\Scripts\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\IoT_Security_Project\AI_Threat_Detection\env\Scripts\python.exe -m pip install --upgrade pip


  Using cached eth_abi-5.1.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached eth_account-0.13.4-py3-none-any.whl.metadata (5.3 kB)
  Using cached eth_hash-0.7.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached eth_typing-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached aiohttp-3.10.10-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websockets-13.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached pyunormalize-16.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached parsimonious-0.10.0-py3-none-any.whl.metadata (25 kB)
  Using cached bitarray-3.0.0


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\IoT_Security_Project\AI_Threat_Detection\env\Scripts\python.exe -m pip install --upgrade pip


In [9]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from web3 import Web3
import json

# Connect to local blockchain
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))

# Load contract ABI and bytecode
artifacts_path = 'C:/Users/PC/IoT_Security_Project/BlockchainProject/artifacts/contracts/IoTData.sol/IoTData.json'
with open(artifacts_path) as artifacts_file:
    artifacts = json.load(artifacts_file)
abi = artifacts['abi']
bytecode = artifacts['bytecode']

# Manually set the contract address (hardcoded)
contract_address = "0x5fbdb2315678afecb367f032d93f642f64180aa3".strip()

# Convert the address to checksum format
contract_address_checksum = w3.to_checksum_address(contract_address)

# Initialize contract instance
contract = w3.eth.contract(address=contract_address_checksum, abi=abi)
# Function to log attack results on the blockchain
def log_attack_result(attack_type, threat_level):
    tx_hash = contract.functions.logAttack(attack_type, threat_level).transact({
        'from': w3.eth.accounts[0]
    })
    w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Logged {attack_type} with threat level {threat_level} to blockchain.")


# List of attack types and benign label
attack_types = ["Bot", "DDoS", "DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris", "FTP-Patator",
                "PortScan", "SSH-Patator"]
benign_type = "BENIGN"

# Initialize an empty list to store results as dictionaries
results = []

# Loop through each attack type
for attack_type in attack_types:
    try:
        # Read the feature importance file
        importance_file = f"{attack_type}_importance.csv"
        importance_data = pd.read_csv(importance_file)
        
        # Select the first 4 features from the feature importance file
        selected_features = importance_data['Feature'][:4].tolist()
        
        # Read the data file
        data_file = f"{attack_type}_vs_{benign_type}.csv"
        data = pd.read_csv(data_file)
        
        # Strip whitespace from column names
        data.columns = data.columns.str.strip()
        
        # Check available columns and selected features
        print(f"\nData file: {data_file}")
        print("Columns available in the dataset:", data.columns.tolist())
        print("Selected features based on importance:", selected_features)

        # Filter only the features that are available in the dataset
        available_features = [feature for feature in selected_features if feature in data.columns]
        
        # Check if 'Label' column exists and if we have at least 2 features to proceed
        if 'Label' not in data.columns or len(available_features) < 2:
            print(f"Warning: Required columns (including Label) missing in {data_file}. Skipping this file.")
            continue
        
        # Select available features and the target column
        selected_data = data[available_features + ['Label']]
        
        # Ensure the Label column is of categorical type (convert if necessary)
        selected_data['Label'] = selected_data['Label'].astype('category')
        
        # Split the data into features (X) and target (y)
        X = selected_data[available_features]
        y = selected_data['Label']
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
        
        # Train Naive Bayes
        nb_model = GaussianNB()
        nb_model.fit(X_train, y_train)
        
        # Train Quadratic Discriminant Analysis
        qda_model = QuadraticDiscriminantAnalysis()
        qda_model.fit(X_train, y_train)
        
        # Train Multi-Layer Perceptron
        mlp_model = MLPClassifier(random_state=42, max_iter=1000, learning_rate_init=0.001)
        mlp_model.fit(X_train, y_train)
        
        # Predictions
        nb_preds = nb_model.predict(X_test)
        qda_preds = qda_model.predict(X_test)
        mlp_preds = mlp_model.predict(X_test)
        
        # Calculate accuracies
        nb_accuracy = accuracy_score(y_test, nb_preds)
        qda_accuracy = accuracy_score(y_test, qda_preds)
        mlp_accuracy = accuracy_score(y_test, mlp_preds)
        
        # Store the results as a dictionary
        result_dict = {
            'Attack Type': attack_type,
            'Naive Bayes Accuracy': nb_accuracy,
            'QDA Accuracy': qda_accuracy,
            'MLP Accuracy': mlp_accuracy
        }
        
        # Append the dictionary to the results list
        results.append(result_dict)
        
    except FileNotFoundError as e:
        print(f"File not found: {e.filename}. Skipping this attack type.")
    except Exception as e:
        print(f"Error processing {attack_type}: {e}")

# Create a DataFrame from the results list and save it
results_df = pd.DataFrame(results)
results_df.to_csv("evaluation_results.csv", index=False)

# Display the DataFrame
print("Evaluation Results:")
print(results_df)

#Adapted from https://github.com/yasakrami/Threat-Detection-in-Cyber-Security-Using-AI


Data file: Bot_vs_BENIGN.csv
Columns available in the dataset: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK 

C:\Program Files\KMSpico\temp\ipykernel_6564\483202837.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['Label'] = selected_data['Label'].astype('category')



Data file: DDoS_vs_BENIGN.csv
Columns available in the dataset: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK

C:\Program Files\KMSpico\temp\ipykernel_6564\483202837.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['Label'] = selected_data['Label'].astype('category')



Data file: DoS GoldenEye_vs_BENIGN.csv
Columns available in the dataset: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Cou

...


Data file: PortScan_vs_BENIGN.csv
Columns available in the dataset: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 

C:\Program Files\KMSpico\temp\ipykernel_6564\483202837.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['Label'] = selected_data['Label'].astype('category')



Data file: SSH-Patator_vs_BENIGN.csv
Columns available in the dataset: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count

C:\Program Files\KMSpico\temp\ipykernel_6564\483202837.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['Label'] = selected_data['Label'].astype('category')
C:\Users\PC\IoT_Security_Project\AI_Threat_Detection\env\Lib\site-packages\sklearn\discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Evaluation Results:
        Attack Type  Naive Bayes Accuracy  QDA Accuracy  MLP Accuracy
0               Bot              0.998870      0.998870      0.999250
1              DDoS              0.932720      0.951301      0.974867
2     DoS GoldenEye              0.955974      0.933303      0.997804
3          DoS Hulk              0.906378      0.906451      0.958199
4  DoS Slowhttptest              0.990503      0.894547      0.998605
5     DoS slowloris              0.967440      0.967379      0.997623
6       FTP-Patator              0.994756      0.998447      0.999896
7          PortScan              0.937511      0.939901      0.987771
8       SSH-Patator              0.993173      0.995212      0.997565
